In [22]:
import re
import pandas as pd
import numpy as np
import json
import os

DATA_IN_PATH='./data_in/'
TRAIN_CLEAN_DATA='train_clean.csv'

train_data=pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

reviews=list(train_data['review'])
sentiments=list(train_data['sentiment'])

sentences=[]
for review in reviews:
    sentences.append(review.split())

In [23]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [24]:
# 학습 시 필요한 하이퍼 파라미터

# 워드 벡터 특징값 수
num_features=300
# 단어에 대한 최소 빈도 수
min_word_count=40
# 프로세스 개수
num_workers=4
# 컨텍스트 윈도우 크기
context=10
# 다운 샘플링 비율
downsampling=1e-3

In [25]:
from gensim.models import word2vec
print("Training model...")
model=word2vec.Word2Vec(sentences,
                       workers=num_workers,
                       size=num_features,
                       min_count=min_word_count,
                       window=context,
                       sample=downsampling)

2020-12-15 09:48:16,633 : INFO : collecting all words and their counts
2020-12-15 09:48:16,634 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-12-15 09:48:16,808 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types


Training model...


2020-12-15 09:48:16,991 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2020-12-15 09:48:17,086 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2020-12-15 09:48:17,086 : INFO : Loading a fresh vocabulary
2020-12-15 09:48:17,126 : INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2020-12-15 09:48:17,127 : INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2020-12-15 09:48:17,147 : INFO : deleting the raw counts dictionary of 74065 items
2020-12-15 09:48:17,149 : INFO : sample=0.001 downsamples 30 most-common words
2020-12-15 09:48:17,150 : INFO : downsampling leaves estimated 2494384 word corpus (94.9% of prior 2627273)
2020-12-15 09:48:17,166 : INFO : estimated required memory for 8160 words and 300 dimensions: 23664000 bytes
2020-12-15 09:48:17,167 : INFO : resetting layer weights
2020-12-15 09:48:18,533 : INFO : t

In [26]:
# 모델의 하이퍼파라미터를 설정한 내용을 모델 이름에 담는다면 나중에 참고하기 좋음
# 모델을 저장하면 Word2Vec.load()로 모델을 다시 사용 가능
model_name="300features_40minwords_10context"
model.save(model_name)

2020-12-15 09:48:36,828 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2020-12-15 09:48:36,829 : INFO : not storing attribute vectors_norm
2020-12-15 09:48:36,830 : INFO : not storing attribute cum_table
2020-12-15 09:48:36,984 : INFO : saved 300features_40minwords_10context


In [27]:
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector=np.zeros((num_features), dtype=np.float32)
    
    num_words=0
    # 어휘사전 준비
    index2word_set=set(model.wv.index2word)
    
    for w in words:
        if w in index2word_set:
            num_words+=1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector=np.add(feature_vector, model[w])
    
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector=np.divide(feature_vector, num_words)
    return feature_vector

In [38]:
def get_dataset(reviews, model, num_features):
    dataset=list()
    
    for s in reviews:
        dataset.append(get_features(s, model, num_features))
    reviewFeatureVecs=np.stack(dataset)
    
    return reviewFeatureVecs

In [39]:
test_data_vecs=get_dataset(sentences, model, num_features)

C:\Users\yuhwa\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [40]:
from sklearn.model_selection import train_test_split

X=test_data_vecs
y=np.array(sentiments)

RANDOM_SEED=42
TEST_SPLIT=0.2

X_train, X_eval, y_train, y_eval=train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [41]:
from sklearn.linear_model import LogisticRegression

lgs=LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

C:\Users\yuhwa\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(class_weight='balanced')

In [42]:
# 검증 데이터로 성능 측정
print("Accuracy: %f" % lgs.score(X_eval, y_eval))

Accuracy: 0.864800


In [43]:
TEST_CLEAN_DATA='test_clean.csv'

test_data=pd.read_csv(DATA_IN_PATH+TEST_CLEAN_DATA)

test_review=list(test_data['review'])

In [44]:
test_sentences=[]
for review in test_data:
    test_sentences.append(review.split())

In [45]:
test_data_vecs=get_dataset(test_sentences, model, num_features)

C:\Users\yuhwa\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [46]:
DATA_OUT_PATH='./data_out/'

test_predicted=lgs.predict(test_data_vecs)

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
ids=list(test_data['id'])
answer_dataset=pd.DataFrame({'id':ids, 'sentiment':test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH+'lgs_answer.csv',index=False)

ValueError: arrays must all be same length

In [79]:
answer_dataset.to_csv(DATA_OUT_PATH+'lgs_answer.csv',index=False)

In [47]:
print(test_predicted)

[0 0]


In [48]:
print(1e-3)

0.001
